# Data Collection, Cleaning, and Concatenation

This notebook collects, joins, and exports all of the initial variables used in the feature set

Imports

In [197]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import finnhub
import yfinance as yf
import talib as ta

Get dates for a ten-year window and convert to UNIX Timestamp integers

In [198]:
# Create datetime objects
start = pd.Timestamp("2011-08-01 23:59:00")
end = pd.Timestamp("2022-08-01 23:59:00")

# Convert datetime objects to UNIX timestamp integers
start_unix = int(pd.Timestamp.timestamp(start))
end_unix = int(pd.Timestamp.timestamp(end))

Get SPY data from FinnHub

In [199]:
# Connect to FinnHub API
load_dotenv()
finnhub_api_key = os.getenv('FINNHUB_API_KEY')
fh = finnhub.Client(api_key=finnhub_api_key)

# Note: price data is already adjusted by Finnhub
spy_ohlcv = fh.stock_candles('SPY', 'D', start_unix, end_unix)
spy_ohlcv = pd.DataFrame(spy_ohlcv)
spy_ohlcv.tail()

,c,h,l,o,s,t,v
2764,390.89,394.06,389.95,393.84,ok,1658793600,52946393
2765,401.04,402.88,394.05,394.36,ok,1658880000,82342106
2766,406.07,406.80,398.15,401.89,ok,1658966400,73966563
2767,411.99,413.03,406.77,407.58,ok,1659052800,87003672
2768,410.77,413.41,408.40,409.15,ok,1659312000,69997471


Check for nulls or missing data

In [200]:
# The `s` column indicates the response status
# Finnhub returns the string "no_data" for missing rows
spy_ohlcv.loc[spy_ohlcv['s'] == 'no_data']

,c,h,l,o,s,t,v


In [201]:
# Check for any null rows
spy_ohlcv.isnull().values.any()

False

Convert the returned UNItechnicals timestamps to date objects and index the date column

In [202]:
spy_ohlcv['date'] = pd.to_datetime(spy_ohlcv['t'], unit='s')
spy_ohlcv = spy_ohlcv.set_index(spy_ohlcv['date'])
spy_ohlcv.tail()

,c,h,l,o,s,t,v,date
date,,,,,,,,
2022-07-26,390.89,394.06,389.95,393.84,ok,1658793600,52946393,2022-07-26
2022-07-27,401.04,402.88,394.05,394.36,ok,1658880000,82342106,2022-07-27
2022-07-28,406.07,406.80,398.15,401.89,ok,1658966400,73966563,2022-07-28
2022-07-29,411.99,413.03,406.77,407.58,ok,1659052800,87003672,2022-07-29
2022-08-01,410.77,413.41,408.40,409.15,ok,1659312000,69997471,2022-08-01


Drop columns and create the features DataFrame 'technicals'

In [203]:
technicals = spy_ohlcv.drop(columns=['s', 't', 'date'])
technicals.columns = ['close', 'high', 'low', 'open', 'volume']

Check dates

In [204]:
display(technicals.shape, technicals.head(), technicals.tail())

(2769, 5)

,close,high,low,open,volume
date,,,,,
2011-08-01,128.78,130.96,127.53,130.84,325790824
2011-08-02,125.49,128.50,125.49,127.81,346653757
2011-08-03,126.17,126.31,123.53,125.66,371029415
2011-08-04,120.26,124.62,120.06,124.42,520721783
2011-08-05,120.08,122.07,116.86,121.76,655561584


,close,high,low,open,volume
date,,,,,
2022-07-26,390.89,394.06,389.95,393.84,52946393
2022-07-27,401.04,402.88,394.05,394.36,82342106
2022-07-28,406.07,406.80,398.15,401.89,73966563
2022-07-29,411.99,413.03,406.77,407.58,87003672
2022-08-01,410.77,413.41,408.40,409.15,69997471


## Create Variables with TA Lib

In [205]:
close = technicals['close']
high = technicals['high']
low = technicals['low']

# Hilbert Transform - Instantaneous Trendline
technicals['trend'] = ta.HT_TRENDLINE(close)

# RSI
technicals['rsi'] = ta.RSI(close, timeperiod=20)

# Stochastic RSI
technicals['rsi_fast_k'], technicals['rsi_fast_d'] = ta.STOCHRSI(
    close, 
    timeperiod=14, 
    fastk_period=5, 
    fastd_period=3, 
    fastd_matype=0
)

# Williams' %R
technicals['williams_r'] = ta.WILLR(high, low, close, timeperiod=14)

# MACD, MACD Signal, MACD Histogram
technicals['macd'], technicals['macd_signal'], technicals['macd_hist'] = ta.MACD(
    close, 
    fastperiod=12, 
    slowperiod=26, 
    signalperiod=9
)

# Moving Averages
technicals['ma_20'] = ta.MA(close, timeperiod=20, matype=0)
technicals['ma_50'] = ta.MA(close, timeperiod=50, matype=0)
technicals['ma_65'] = ta.MA(close, timeperiod=65, matype=0)
technicals['ma_200'] = ta.MA(close, timeperiod=200, matype=0)

# Bollinger Bands
technicals['bb_upp'], technicals['bb_mid'], technicals['bb_low'] = ta.BBANDS(
    close, 
    timeperiod=5, 
    nbdevup=2, 
    nbdevdn=2, 
    matype=0
)

# Percentage Price Oscillator
technicals['ppo'] = ta.PPO(close, fastperiod=12, slowperiod=26, matype=0)

# Momentum
technicals['mom'] = ta.MOM(close, timeperiod=10)

# Rate of Change
technicals['roc'] = ta.ROC(close, timeperiod=10)

# Exponential Moving Averages
technicals['ema_20'] = ta.EMA(close, timeperiod=20)
technicals['ema_50'] = ta.EMA(close, timeperiod=50)
technicals['ema_65'] = ta.EMA(close, timeperiod=65)
technicals['ema_200'] = ta.EMA(close, timeperiod=200)

Slice the data from Aug 1, 2012 onward

In [206]:
# Remove all data before Aug 1, 2012 for an exact ten years
technicals = technicals.loc['2012-08-01':]
display(technicals.shape, technicals.head())

(2516, 27)

,close,high,low,open,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,bb_upp,bb_mid,bb_low,ppo,mom,roc,ema_20,ema_50,ema_65,ema_200
date,,,,,,,,,,,,,,,,,,,,,
2012-08-01,137.590,138.73,137.4000,138.70,138293740,136.387994,55.587587,36.624310,70.086458,-27.722301,...,139.607390,137.7650,135.922610,0.693206,0.220,0.160151,136.139105,135.287401,135.135725,130.535144
2012-08-02,136.640,137.57,135.5800,136.55,199556580,136.561699,52.970383,0.000000,36.753125,-42.780155,...,139.387508,137.8590,136.330492,0.577563,-1.090,-0.791403,136.186809,135.340444,135.181310,130.595888
2012-08-03,139.349,139.64,136.6794,138.56,157824975,136.708467,58.793873,97.029563,44.551291,-4.402421,...,139.866511,137.9938,136.121089,0.523204,2.879,2.109621,136.487970,135.497642,135.307603,130.682984
2012-08-06,139.620,140.17,139.5600,139.72,86327738,136.843608,59.324250,100.000000,65.676521,-7.703081,...,140.441135,138.1818,135.922465,0.445066,4.530,3.353320,136.786259,135.659303,135.438282,130.771910
2012-08-07,140.320,140.92,140.0300,140.18,109545089,137.054508,60.699634,100.000000,99.009854,-7.604563,...,141.441336,138.7038,135.966264,0.560776,6.395,4.775061,137.122805,135.842076,135.586213,130.866915


Remove high, low, and open from our initial variable set

In [207]:
technicals = technicals.drop(columns=['high', 'low', 'open'])

Check for nulls

In [208]:
technicals[technicals.isnull().any(axis=1)]

,close,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,macd,macd_signal,macd_hist,...,bb_upp,bb_mid,bb_low,ppo,mom,roc,ema_20,ema_50,ema_65,ema_200
date,,,,,,,,,,,,,,,,,,,,,


## Get Bond Data from CSVs

In [209]:
# Import bond data CSVs
# We are using only the % change column of each bond
bond_3mt = pd.read_csv('../csv/us-3mt-bond.csv', usecols=['Date', 'Change %'], index_col='Date', parse_dates=True, infer_datetime_format=True)
bond_2yr = pd.read_csv('../csv/us-2yr-bond.csv', usecols=['Date', 'Change %'], index_col='Date', parse_dates=True, infer_datetime_format=True)
bond_5yr = pd.read_csv('../csv/us-5yr-bond.csv', usecols=['Date', 'Change %'], index_col='Date', parse_dates=True, infer_datetime_format=True)
bond_10yr = pd.read_csv('../csv/us-10yr-bond.csv', usecols=['Date', 'Change %'], index_col='Date', parse_dates=True, infer_datetime_format=True)

bond_3mt.columns = ['bond_3mt']
bond_2yr.columns = ['bond_2yr']
bond_5yr.columns = ['bond_5yr']
bond_10yr.columns = ['bond_10yr']

In [210]:
# Join bonds
bonds = pd.concat([bond_3mt, bond_2yr, bond_5yr, bond_10yr], join='inner', axis='columns')

# Remove percentage symbol from each row and convert to decimal value
bonds = bonds.apply(
    lambda bond: bond.str.rstrip("%").replace(',','', regex=True).astype(float) / 100
)

bonds.head()

,bond_3mt,bond_2yr,bond_5yr,bond_10yr
Date,,,,
2022-08-01,0.0651,-0.0160,-0.0279,-0.0384
2022-07-31,0.0022,0.0105,0.0081,0.0048
2022-07-29,-0.0054,0.0077,-0.0062,-0.0048
2022-07-28,-0.0265,-0.0387,-0.0487,-0.0411
2022-07-27,-0.0312,-0.0246,-0.0221,-0.0065


Check for null values

In [211]:
bonds[bonds.isnull().any(axis=1)]

,bond_3mt,bond_2yr,bond_5yr,bond_10yr
Date,,,,


Check shape and dates

In [212]:
display(bonds.shape, bonds)

(3012, 4)

,bond_3mt,bond_2yr,bond_5yr,bond_10yr
Date,,,,
2022-08-01,0.0651,-0.0160,-0.0279,-0.0384
2022-07-31,0.0022,0.0105,0.0081,0.0048
2022-07-29,-0.0054,0.0077,-0.0062,-0.0048
2022-07-28,-0.0265,-0.0387,-0.0487,-0.0411
2022-07-27,-0.0312,-0.0246,-0.0221,-0.0065
...,...,...,...,...
2012-08-07,0.0495,0.1167,0.0846,0.0403
2012-08-06,0.2454,0.0017,-0.0240,-0.0019
2012-08-03,-0.1099,0.0573,0.0900,0.0609


## Get features from Yahoo!

In [213]:
# Row 1: USD/x Currency pairs
# Row 2: US Indices
# Row 3: Futures for S&P 500, Dow Jones Industrial, Nasdaq, Russell 2000, US Treasury Bonds, Oil, Natural Gas, Gold, Silver, Copper, Wheat, Corn, Soybean, 
yf_tickers = \
"CAD=X CNY=X HKD=X AUD=X JPY=X EUR=X \
^GSPC ^IXIC ^DJI ^DJT ^RUT ^VIX ^FTSE ^HSI ^N225 \
ES=F YM=F NQ=F ZB=F ZN=F ZT=F CL=F NG=F GC=F SI=F HG=F KE=F ZC=F ZF=F ZS=F \
AAPL MSFT AMZN TSLA UNH NVDA JNJ XOM PG V JPM WMT KO"

# Fetch market data and use only the adjusted close
markets_ohlcv = yf.download(yf_tickers, start='2012-08-01', end='2022-08-02')
markets = markets_ohlcv['Adj Close'].drop(index='2022-08-02')
markets.tail()

[*********************100%***********************]  43 of 43 completed


,AAPL,AMZN,AUD=X,CAD=X,CL=F,CNY=X,ES=F,EUR=X,GC=F,HG=F,...,ZT=F,^DJI,^DJT,^FTSE,^GSPC,^HSI,^IXIC,^N225,^RUT,^VIX
Date,,,,,,,,,,,,,,,,,,,,,
2022-07-26,151.600006,114.809998,1.438600,1.28507,94.980003,6.7495,3923.25,0.97800,1717.699951,3.3825,...,104.976562,31761.539062,13614.139648,7306.299805,3921.050049,20905.880859,11562.570312,27655.210938,1805.250000,24.690001
2022-07-27,156.789993,120.970001,1.439390,1.28737,97.260002,6.7623,4024.50,0.98721,1719.099976,3.4295,...,105.101562,32197.589844,13847.009766,7348.200195,4023.610107,20670.039062,12032.419922,27715.750000,1848.339966,23.240000
2022-07-28,157.350006,122.279999,1.429613,1.28171,96.419998,6.7574,4073.50,0.97950,1750.300049,3.4750,...,105.281250,32529.630859,14275.299805,7345.299805,4072.429932,20622.679688,12162.589844,27815.480469,1873.030029,22.330000
2022-07-29,162.509995,134.949997,1.428400,1.28070,98.620003,6.7458,4133.50,0.98113,1762.900024,3.5840,...,105.230469,32845.128906,14609.000000,7423.399902,4130.290039,20156.509766,12390.690430,27801.640625,1885.229980,21.330000
2022-08-01,161.509995,135.389999,1.434470,1.28156,93.889999,6.7432,4120.50,0.97960,1769.000000,3.5520,...,105.203125,32798.398438,14634.089844,7413.399902,4118.629883,20165.839844,12368.980469,27993.349609,1883.310059,22.840000


In [214]:
markets.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2609 entries, 2012-08-01 to 2022-08-01
Freq: B
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2516 non-null   float64
 1   AMZN    2516 non-null   float64
 2   AUD=X   2604 non-null   float64
 3   CAD=X   2604 non-null   float64
 4   CL=F    2516 non-null   float64
 5   CNY=X   2604 non-null   float64
 6   ES=F    2516 non-null   float64
 7   EUR=X   2604 non-null   float64
 8   GC=F    2514 non-null   float64
 9   HG=F    2515 non-null   float64
 10  HKD=X   2604 non-null   float64
 11  JNJ     2516 non-null   float64
 12  JPM     2516 non-null   float64
 13  JPY=X   2605 non-null   float64
 14  KE=F    2514 non-null   float64
 15  KO      2516 non-null   float64
 16  MSFT    2516 non-null   float64
 17  NG=F    2516 non-null   float64
 18  NQ=F    2516 non-null   float64
 19  NVDA    2516 non-null   float64
 20  PG      2516 non-null   float64
 21  SI=F    251

Format column Names

In [215]:
yf_columns = [
    'stk_aapl', 'stk_amzn', 'usd_aud', 'usd_cad', 'fut_crude', 'usd_cny', 'fut_sp',
    'usd_eur', 'fut_gold', 'fut_copper', 'usd_hkd', 'stk_jnj', 'stk_jpm', 'usd_jpy', 'fut_wheat', 'stk_ko', 'stk_msft', 
    'fut_natgas', 'fut_nasdaq', 'stk_nvda', 'stk_pg', 'fut_silver', 'stk_tsla', 'stk_unh', 'stk_visa', 'stk_wmt', 'stk_xom',
    'fut_dji', 'fut_us_tbond', 'fut_corn', 'fut_5yr_tnote', 'fut_10yr_tnote', 'fut_soybean', 'fut_2yr_tnote',
    'idx_dji', 'idx_djt', 'idx_ftse', 'idx_gspc', 'idx_hsi', 'idx_ixic', 'idx_n225', 'idx_rut', 'idx_vix'
]

markets.columns = yf_columns
markets = markets.reindex(columns=np.sort(yf_columns))

display(markets, markets.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2609 entries, 2012-08-01 to 2022-08-01
Freq: B
Data columns (total 43 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   fut_10yr_tnote  2514 non-null   float64
 1   fut_2yr_tnote   2516 non-null   float64
 2   fut_5yr_tnote   2516 non-null   float64
 3   fut_copper      2515 non-null   float64
 4   fut_corn        2512 non-null   float64
 5   fut_crude       2516 non-null   float64
 6   fut_dji         2516 non-null   float64
 7   fut_gold        2514 non-null   float64
 8   fut_nasdaq      2516 non-null   float64
 9   fut_natgas      2516 non-null   float64
 10  fut_silver      2513 non-null   float64
 11  fut_soybean     2514 non-null   float64
 12  fut_sp          2516 non-null   float64
 13  fut_us_tbond    2513 non-null   float64
 14  fut_wheat       2514 non-null   float64
 15  idx_dji         2516 non-null   float64
 16  idx_djt         2516 non-null   float64
 17  idx_fts

,fut_10yr_tnote,fut_2yr_tnote,fut_5yr_tnote,fut_copper,fut_corn,fut_crude,fut_dji,fut_gold,fut_nasdaq,fut_natgas,...,stk_unh,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy
Date,,,,,,,,,,,,,,,,,,,,,
2012-08-01,134.156250,110.273438,124.546875,3.3765,800.50,88.910004,12923.0,1603.699951,2625.0,3.171,...,43.832554,29.716949,58.796177,57.188221,0.953740,1.00320,6.3618,0.81340,7.75376,78.120003
2012-08-02,134.625000,110.281250,124.695312,3.2925,794.00,87.129997,12831.0,1587.400024,2618.5,2.920,...,43.738400,30.138947,59.139614,56.510487,0.955110,1.00472,6.3688,0.81690,7.75500,78.379997
2012-08-03,133.906250,110.257812,124.382812,3.3710,810.00,91.400002,13055.0,1606.000000,2671.0,2.877,...,44.174931,30.537621,59.538914,57.609356,0.956130,1.00705,6.3676,0.82100,7.75426,78.220001
2012-08-06,134.109375,110.265625,124.531250,3.3930,803.00,92.199997,13067.0,1612.900024,2686.5,2.908,...,43.652798,30.549292,59.323288,57.543537,0.947060,1.00057,6.3715,0.80460,7.75430,78.610001
2012-08-07,133.578125,110.218750,124.273438,3.4450,796.00,93.669998,13119.0,1609.699951,2710.0,2.964,...,44.970951,30.402401,59.091682,57.852825,0.946700,1.00060,6.3740,0.80710,7.75500,78.199997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-26,119.859375,104.976562,112.906250,3.3825,597.00,94.980003,31732.0,1717.699951,12112.5,8.993,...,531.590027,212.490005,121.980003,89.629997,1.438600,1.28507,6.7495,0.97800,7.84913,136.431000
2022-07-27,120.421875,105.101562,113.296875,3.4295,600.25,97.260002,32172.0,1719.099976,12619.0,8.687,...,534.609985,210.470001,126.589996,91.570000,1.439390,1.28737,6.7623,0.98721,7.84926,136.970001
2022-07-28,120.984375,105.281250,113.718750,3.4750,615.00,96.419998,32490.0,1750.300049,12737.5,8.134,...,541.489990,211.350006,129.750000,92.639999,1.429613,1.28171,6.7574,0.97950,7.84910,136.110992


None

Check for nulls

In [216]:
markets[markets.isna().any(axis=1)]

,fut_10yr_tnote,fut_2yr_tnote,fut_5yr_tnote,fut_copper,fut_corn,fut_crude,fut_dji,fut_gold,fut_nasdaq,fut_natgas,...,stk_unh,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy
Date,,,,,,,,,,,,,,,,,,,,,
2012-08-27,133.812500,110.210938,124.468750,3.4845,794.50,95.470001,13107.0,1672.400024,2782.75,2.653,...,46.742733,29.814079,58.214832,58.109581,0.961090,0.99208,6.3548,0.79972,7.75610,78.730003
2012-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.972940,0.98600,6.3486,0.79510,7.75590,78.290001
2012-09-17,133.312500,110.218750,124.437500,3.8100,748.00,96.620003,13541.0,1767.699951,2852.00,2.865,...,46.820343,31.264370,59.411251,60.871346,0.948890,0.97159,6.3153,0.76240,7.75106,78.333000
2012-10-01,133.578125,110.265625,124.664062,3.7930,756.75,92.480003,13437.0,1780.500000,2788.25,3.480,...,48.521976,31.932308,59.459431,60.798515,0.965680,0.98420,6.2848,0.78070,7.75330,77.959000
2012-10-02,133.640625,110.265625,124.703125,3.8070,758.25,91.889999,13411.0,1772.699951,2795.25,3.531,...,48.814159,31.761822,59.218548,60.745495,0.965160,0.98190,6.2848,0.77590,7.75420,78.041000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-03,119.046875,105.808594,112.742188,4.4775,727.00,118.870003,32888.0,1845.400024,12551.00,8.523,...,483.884369,212.649994,125.320000,99.089996,1.375705,1.25698,6.6592,0.93014,7.84476,129.917999
2022-06-20,116.515625,104.695312,110.882812,3.9650,NaN,110.269997,30148.0,NaN,11420.75,6.694,...,NaN,NaN,NaN,NaN,1.438228,1.30005,6.7157,0.95316,7.84968,135.276001
2022-07-01,119.265625,105.187500,112.835938,3.6190,754.50,108.430000,31061.0,1798.900024,11611.25,5.730,...,517.400024,199.179993,122.629997,87.550003,1.449780,1.28737,6.6982,0.95441,7.84710,135.785995


#### 😱 *null values!*

Since we have existing data alongside of nulls, we want to fill the gaps rather than delete existing data

We will use `df.ffill()` to forward-fill the nulls 

(Back-filling would introduce leakage from future data, which we don't want)

In [217]:
markets = markets.fillna(method='ffill')
markets[markets.isna().any(axis=1)]

,fut_10yr_tnote,fut_2yr_tnote,fut_5yr_tnote,fut_copper,fut_corn,fut_crude,fut_dji,fut_gold,fut_nasdaq,fut_natgas,...,stk_unh,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy
Date,,,,,,,,,,,,,,,,,,,,,


Check shape and dates

In [218]:
display(markets.shape, markets)

(2609, 43)

,fut_10yr_tnote,fut_2yr_tnote,fut_5yr_tnote,fut_copper,fut_corn,fut_crude,fut_dji,fut_gold,fut_nasdaq,fut_natgas,...,stk_unh,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy
Date,,,,,,,,,,,,,,,,,,,,,
2012-08-01,134.156250,110.273438,124.546875,3.3765,800.50,88.910004,12923.0,1603.699951,2625.0,3.171,...,43.832554,29.716949,58.796177,57.188221,0.953740,1.00320,6.3618,0.81340,7.75376,78.120003
2012-08-02,134.625000,110.281250,124.695312,3.2925,794.00,87.129997,12831.0,1587.400024,2618.5,2.920,...,43.738400,30.138947,59.139614,56.510487,0.955110,1.00472,6.3688,0.81690,7.75500,78.379997
2012-08-03,133.906250,110.257812,124.382812,3.3710,810.00,91.400002,13055.0,1606.000000,2671.0,2.877,...,44.174931,30.537621,59.538914,57.609356,0.956130,1.00705,6.3676,0.82100,7.75426,78.220001
2012-08-06,134.109375,110.265625,124.531250,3.3930,803.00,92.199997,13067.0,1612.900024,2686.5,2.908,...,43.652798,30.549292,59.323288,57.543537,0.947060,1.00057,6.3715,0.80460,7.75430,78.610001
2012-08-07,133.578125,110.218750,124.273438,3.4450,796.00,93.669998,13119.0,1609.699951,2710.0,2.964,...,44.970951,30.402401,59.091682,57.852825,0.946700,1.00060,6.3740,0.80710,7.75500,78.199997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-26,119.859375,104.976562,112.906250,3.3825,597.00,94.980003,31732.0,1717.699951,12112.5,8.993,...,531.590027,212.490005,121.980003,89.629997,1.438600,1.28507,6.7495,0.97800,7.84913,136.431000
2022-07-27,120.421875,105.101562,113.296875,3.4295,600.25,97.260002,32172.0,1719.099976,12619.0,8.687,...,534.609985,210.470001,126.589996,91.570000,1.439390,1.28737,6.7623,0.98721,7.84926,136.970001
2022-07-28,120.984375,105.281250,113.718750,3.4750,615.00,96.419998,32490.0,1750.300049,12737.5,8.134,...,541.489990,211.350006,129.750000,92.639999,1.429613,1.28171,6.7574,0.97950,7.84910,136.110992


## Join DataFrames

In [219]:
X = technicals.join(bonds, on=technicals.index)
X = X.join(markets, on=X.index)
X

,close,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,macd,macd_signal,macd_hist,...,stk_unh,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy
date,,,,,,,,,,,,,,,,,,,,,
2012-08-01,137.590,138293740,136.387994,55.587587,36.624310,70.086458,-27.722301,0.867818,0.685636,0.182183,...,43.832554,29.716949,58.796177,57.188221,0.953740,1.00320,6.3618,0.81340,7.75376,78.120003
2012-08-02,136.640,199556580,136.561699,52.970383,0.000000,36.753125,-42.780155,0.798882,0.708285,0.090597,...,43.738400,30.138947,59.139614,56.510487,0.955110,1.00472,6.3688,0.81690,7.75500,78.379997
2012-08-03,139.349,157824975,136.708467,58.793873,97.029563,44.551291,-4.402421,0.951871,0.757002,0.194869,...,44.174931,30.537621,59.538914,57.609356,0.956130,1.00705,6.3676,0.82100,7.75426,78.220001
2012-08-06,139.620,86327738,136.843608,59.324250,100.000000,65.676521,-7.703081,1.082505,0.822103,0.260402,...,43.652798,30.549292,59.323288,57.543537,0.947060,1.00057,6.3715,0.80460,7.75430,78.610001
2012-08-07,140.320,109545089,137.054508,60.699634,100.000000,99.009854,-7.604563,1.228357,0.903354,0.325004,...,44.970951,30.402401,59.091682,57.852825,0.946700,1.00060,6.3740,0.80710,7.75500,78.199997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-26,390.890,52946393,385.898000,49.785606,0.000000,34.261989,-31.880577,1.139625,-0.797274,1.936899,...,531.590027,212.490005,121.980003,89.629997,1.438600,1.28507,6.7495,0.97800,7.84913,136.431000
2022-07-27,401.040,82342106,386.804389,55.163433,98.608208,42.891288,-5.778894,1.935160,-0.250787,2.185947,...,534.609985,210.470001,126.589996,91.570000,1.439390,1.28737,6.7623,0.98721,7.84926,136.970001
2022-07-28,406.070,73966563,387.952778,57.535789,100.000000,66.202736,-2.041387,2.937644,0.386899,2.550744,...,541.489990,211.350006,129.750000,92.639999,1.429613,1.28171,6.7574,0.97950,7.84910,136.110992


## Add Primitive Variables

In [220]:
X['day_of_week'] = X.index.dayofweek
# X['week_of_year'] = X.index.isocalendar().week


### DataFrame Overview

In [221]:
display(X.shape, X)

(2516, 72)

,close,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,macd,macd_signal,macd_hist,...,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy,day_of_week
date,,,,,,,,,,,,,,,,,,,,,
2012-08-01,137.590,138293740,136.387994,55.587587,36.624310,70.086458,-27.722301,0.867818,0.685636,0.182183,...,29.716949,58.796177,57.188221,0.953740,1.00320,6.3618,0.81340,7.75376,78.120003,2
2012-08-02,136.640,199556580,136.561699,52.970383,0.000000,36.753125,-42.780155,0.798882,0.708285,0.090597,...,30.138947,59.139614,56.510487,0.955110,1.00472,6.3688,0.81690,7.75500,78.379997,3
2012-08-03,139.349,157824975,136.708467,58.793873,97.029563,44.551291,-4.402421,0.951871,0.757002,0.194869,...,30.537621,59.538914,57.609356,0.956130,1.00705,6.3676,0.82100,7.75426,78.220001,4
2012-08-06,139.620,86327738,136.843608,59.324250,100.000000,65.676521,-7.703081,1.082505,0.822103,0.260402,...,30.549292,59.323288,57.543537,0.947060,1.00057,6.3715,0.80460,7.75430,78.610001,0
2012-08-07,140.320,109545089,137.054508,60.699634,100.000000,99.009854,-7.604563,1.228357,0.903354,0.325004,...,30.402401,59.091682,57.852825,0.946700,1.00060,6.3740,0.80710,7.75500,78.199997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-26,390.890,52946393,385.898000,49.785606,0.000000,34.261989,-31.880577,1.139625,-0.797274,1.936899,...,212.490005,121.980003,89.629997,1.438600,1.28507,6.7495,0.97800,7.84913,136.431000,1
2022-07-27,401.040,82342106,386.804389,55.163433,98.608208,42.891288,-5.778894,1.935160,-0.250787,2.185947,...,210.470001,126.589996,91.570000,1.439390,1.28737,6.7623,0.98721,7.84926,136.970001,2
2022-07-28,406.070,73966563,387.952778,57.535789,100.000000,66.202736,-2.041387,2.937644,0.386899,2.550744,...,211.350006,129.750000,92.639999,1.429613,1.28171,6.7574,0.97950,7.84910,136.110992,3


Check for null values

In [222]:
X[X.isnull().any(axis=1)]

,close,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,macd,macd_signal,macd_hist,...,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy,day_of_week
date,,,,,,,,,,,,,,,,,,,,,
2012-10-08,145.642,78415562,144.708859,58.967783,54.360463,84.786821,-36.208531,0.801083,0.928451,-0.127367,...,32.394733,60.422997,61.381298,0.98324,0.97724,6.3241,0.76830,7.75226,78.702003,0
2012-11-12,138.265,97677409,141.417193,37.726995,7.165676,3.628884,-88.411669,-1.335464,-0.859280,-0.476184,...,33.452663,58.198765,58.193497,0.96200,1.00000,6.2447,0.78592,7.75070,79.459999,0
2016-11-11,216.420,100552732,212.902219,56.006717,81.395959,93.798653,-19.033233,0.115915,-0.551034,0.666948,...,78.827919,63.429581,64.587990,1.31430,1.34704,6.8017,0.91830,7.75590,106.856003,4
2020-10-12,352.430,80388533,334.376000,61.476107,100.000000,100.000000,-4.646406,2.523982,0.398313,2.125669,...,204.108719,140.419357,31.186638,1.38416,1.31300,6.6937,0.84625,7.74998,105.632004,0


One final forward-fill

In [223]:
X = X.fillna(method='ffill')
X[X.isnull().any(axis=1)]

,close,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,macd,macd_signal,macd_hist,...,stk_visa,stk_wmt,stk_xom,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_jpy,day_of_week
date,,,,,,,,,,,,,,,,,,,,,


#### All of the data are joined, and there are no null values

In [225]:
# Export to CSV
# X.to_csv('../csv/X.csv')

# END